In [ ]:
import random
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification







In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/My Drive/data.csv')
data.head()

,content,morality
0,所以當佢兩隊都係作客咁睇,False
1,今日工作壓力勁大覺得個病又起,True
2,其實只要讓我耿耿某人思憶早閃閃發光,False
3,仲有d假膠尤其是來自波台既係到亂作答案,False
4,可以睇我profile我開過post講但係真係好撚恐怖,True


In [ ]:
  opt = tf.keras.optimizers.Adam(1e-5)
  print(f"this is opt: {opt}")

this is opt: <tf_keras.src.optimizers.adam.Adam object at 0x7dfcf5c9a350>


In [ ]:
# suicide_299 fulltext seed6 50 epoch

# Define the number of folds (k)
k = 5
# # Create an empty DataFrame to store the mean AUC values
# mean_auc_df = pd.DataFrame(columns=['Seed', 'Mean AUC'])

# List of seed values to iterate over (1 to 60)
seed_values = list(range(6,7))

for seed in seed_values:
    print(f"Running with seed: {seed}")
    data = data
    # Set the seed for Python random module
    random.seed(seed)

    # Set the seed for NumPy
    np.random.seed(seed)
    # optimizer = optimizers.Adam(learning_rate=1e-5)
    # Set the seed for TensorFlow
    tf.random.set_seed(seed)
    # Initialize the stratified k-fold cross-validator
    kf = StratifiedKFold(n_splits=k,random_state=seed, shuffle=True) # ensure the shuffling is the reproducible
    # Load the pre-trained BERT model and tokenizer
    model_name = "bert-base-uncased"
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=2)
    # Define a loss function and metrics
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy("accuracy")]
    # Define an optimizer
    opt = tf.keras.optimizers.Adam(1e-5)
    print(f"this is opt: {opt}")
    # optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    # optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    # tf.keras.optimizers.Optimizer(
    # name='adam',
    # learning_rate=0.001,
    # weight_decay=0,
    # clipnorm=None,
    # clipvalue=None,
    # global_clipnorm=None,
    # use_ema=False,
    # ema_momentum=0.99,
    # ema_overwrite_frequency=None,
    # jit_compile=True,
    # )
    #opt=optimizers.Adam(learning_rate=1e-5)
    # Split the dataset into features and target arrays
    texts = data['content'].values
    labels = data['morality'].values
    fold_results = []
    # i=1
    # Iterate through the k folds
    for fold, (train_val_index, test_index) in enumerate(kf.split(texts, labels)):
      # Set the seed for each fold (to ensure the reproducibility of each fold, these are fold-specific seeds)
      tf.random.set_seed(seed)
      np.random.seed(seed)
      random.seed(seed)
      # Split train_val set to train and validation sets in 8:2 ratio
      train_index, valid_index = train_test_split(train_val_index, test_size=0.2, stratify=labels[train_val_index], random_state=seed) #ensure the splitting is reproducible
      print(f"Fold {fold + 1}/{k}")
      # Create train, validation, and test datasets for this fold
      train_texts, train_labels = texts[train_index], labels[train_index]
      valid_texts, valid_labels = texts[valid_index], labels[valid_index]
      test_texts, test_labels = texts[test_index], labels[test_index]
      # Tokenize the input texts for the fold
      train_inputs = tokenizer(list(train_texts), padding=True, truncation=True, return_tensors="tf")
      valid_inputs = tokenizer(list(valid_texts), padding=True, truncation=True, return_tensors="tf")
      test_inputs = tokenizer(list(test_texts), padding=True, truncation=True, return_tensors="tf")
      # Define TensorFlow Datasets for training and validation
      train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_inputs), train_labels))
      valid_dataset = tf.data.Dataset.from_tensor_slices((dict(valid_inputs), valid_labels))
      test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_inputs), test_labels))
      # Compile the model for this fold
      model.compile(optimizer=opt, loss=loss_fn, metrics=metrics)
      # Train the model for this fold
      batch_size = 8
      num_epochs = 50
      model.fit(train_dataset.shuffle(100).batch(batch_size), epochs=num_epochs, validation_data=valid_dataset.batch(batch_size))
      # Evaluate the model on the validation data for this fold
      loss, accuracy = model.evaluate(valid_dataset.batch(batch_size))
      print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")
      model_save_path = f"/content/sample_data/model/model{fold + 1}"
      model.save_pretrained(model_save_path)
      # Create a DataFrame for test data
      test_data_df = pd.DataFrame({'Case':test_index,'text': test_texts, 'label': test_labels})
      # Save the test data to a CSV file
      test_data_csv_path = f"/content/sample_data/dataset/test{fold + 1}.csv"
      test_data_df.to_csv(test_data_csv_path, index=False)
    sleep(2)
  # #testing
  #   model_name = "bert-base-uncased"
  #   tokenizer = BertTokenizer.from_pretrained(model_name)
    # k = 5  # Number of folds
    final_results = []
    for fold in range(k):
        # Load the model for the current fold
        model_path = f"/content/sample_data/model/model{fold + 1}"
        loaded_model = TFBertForSequenceClassification.from_pretrained(model_path)
        # Load the test data for the current fold
        data_path = f"/content/sample_data/dataset/test{fold + 1}.csv"
        data = pd.read_csv(data_path)

        # Prepare to store results
        predictions = []
        predicted_probabilities_list0 = []
        predicted_probabilities_list1 = []
        # predicted_probabilities_list2 = []
        # predicted_probabilities_list3 = []
        # Iterate over each row in the data
        for text in data['text']:
            test_inputs = tokenizer(text, padding=True, truncation=True, return_tensors="tf")
            prediction = loaded_model(test_inputs)
            predicted_probabilities = tf.nn.softmax(prediction.logits, axis=1).numpy()
            predicted_class = np.argmax(predicted_probabilities, axis=1)

            predictions.append(predicted_class[0])
            predicted_probabilities_list0.append(predicted_probabilities[0][0])
            predicted_probabilities_list1.append(predicted_probabilities[0][1])
            # predicted_probabilities_list2.append(predicted_probabilities[0][2])
            # predicted_probabilities_list3.append(predicted_probabilities[0][3])
        # Store results in the DataFrame
        data['predicted_class'] = predictions
        data['Fold_Model'] = fold + 1
        data['prob_class_0'] = predicted_probabilities_list0
        data['prob_class_1'] = predicted_probabilities_list1
        # data['prob_class_2'] = predicted_probabilities_list2
        # data['prob_class_3'] = predicted_probabilities_list3

        # Append the results of this fold to the final results
        final_results.append(data)

    # Concatenate all results into a single DataFrame
    final_result_data = pd.concat(final_results)
    sleep(2)
    final_result_data.to_csv(f"/content/drive/MyDrive/suicide/result_suicide_colab_seed6_full_text_epoch50.csv")
    sleep(2)
    # datasets = []
    # for i in range(1, 15):
    file_path = f"/content/drive/MyDrive/suicide/result_suicide_colab_seed6_full_text_epoch50.csv"
    dataset = pd.read_csv(file_path)
    # datasets.append(dataset)
    # merged_df = pd.concat(datasets, ignore_index=True)
    # merged_df['label'].value_counts()
    # final_merged_dfN=merged_df
    label_col = 'label'
    prob_columns = ['prob_class_0', 'prob_class_1']  # Add more if needed
    def calculate_roc_auc(data, label_col, prob_columns, class_label):
        # Drop rows with NaN values in label or probability columns
        data = data.dropna(subset=[label_col] + prob_columns)
        actual = (data[label_col] == class_label).astype(int)
        predicted_prob = data[prob_columns].loc[:, f'prob_class_{class_label}']
        fpr, tpr, _ = roc_curve(actual, predicted_prob)
        auc_score = auc(fpr, tpr)
        return fpr, tpr, auc_score

    # Calculate mean ROC AUC for all classes
    mean_fpr = np.linspace(0, 1, 100)
    mean_tpr = np.zeros_like(mean_fpr)
    mean_auc_values = []

    # Iterate over each class
    for class_label in range(len(prob_columns)):
        fpr, tpr, auc_score = calculate_roc_auc(dataset, label_col, prob_columns, class_label)
        mean_tpr += np.interp(mean_fpr, fpr, tpr)
        mean_auc_values.append(auc_score)
        print(f'AUC for Class {class_label}: {auc_score}')

    # Calculate mean TPR and AUC
    mean_tpr /= len(prob_columns)
    mean_auc = np.mean(mean_auc_values)
    print(f'Mean AUC across all classes: {mean_auc}')

    # # Save mean AUC to DataFrame
    # mean_auc_df = mean_auc_df.append({'Seed': seed, 'Mean AUC': mean_auc}, ignore_index=True)

    # # Plot mean ROC curve
    # plt.figure(figsize=(8, 8))
    # plt.plot(mean_fpr, mean_tpr, color='darkorange', lw=2, label=f'Bert AUC = {mean_auc:.2f}')
    # plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    # plt.xlabel('False Positive Rate')
    # plt.ylabel('True Positive Rate')
    # plt.title('Mean ROC Curve For All Data in multilabel')
    # plt.legend(loc="lower right")
    # # Adjust layout if necessary
    # plt.tight_layout()
    # plt.savefig(f"/content/drive/MyDrive/299_binary_label2/auc/auc_{seed}.png")
    # sleep(2)

# # Save mean AUC DataFrame to CSV file
# mean_auc_df.to_csv('/content/drive/MyDrive/full_text_depression/auc_299_binary_fulltext.csv', index=False)

Running with seed: 6


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


this is opt: <tf_keras.src.optimizers.adam.Adam object at 0x7dfc382d5c30>
Fold 1/5
Epoch 1/50


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
240/240 [==============================] - 16744s 69s/step - loss: 0.6478 - accuracy: 0.6073 - val_loss: 0.6287 - val_accuracy: 0.6750
Epoch 2/50
110/240 [============>.................] - ETA: 2:20:28 - loss: 0.5956 - accuracy: 0.6955